**The Fine-Tuning was done in a different notebook, The tuned model adapters were saved to huggingface which we will use here for inference. Feel free to use your own tuned adapters, We will update the adapters as we train the model on more data and do RLHF.**

# **Loading the saved model**

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git  -U
!pip install -q bitsandbytes accelerate
!pip install -q git+https://github.com/huggingface/peft.git  -U
!pip install -q -U datasets
!pip install -q trl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.1 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [ ]:
%pip install datasets==2.16.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 1.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.2.0
    Uninstalling fsspec-2024.2.0:
      Successfully uninstalled fsspec-2024.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset

2024-03-14 11:32:14.245360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 11:32:14.245507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 11:32:14.399982: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
basemodel="/kaggle/input/mistral/pytorch/7b-v0.1-hf/1"
tunedmodel="KaggleMasterX/mistral7b_0403"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(basemodel)

model = AutoModelForCausalLM.from_pretrained(
    basemodel,
    quantization_config=bnb_config,
    device_map= "auto",
    low_cpu_mem_usage = True,
)

# KV cache is useless during training(Finetune), It only works for inference.
model.config.use_cache = False # enable this in inference mode
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
from peft import *
peft_model_id = tunedmodel
config = PeftConfig.from_pretrained(tunedmodel)
model = AutoModelForCausalLM.from_pretrained(
    basemodel,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(basemodel)
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
model

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_pr

# **Replace abstract as your desired abstract and the run the cells after it**

In [ ]:
instruction="Craft an intelligent, clear, insightful, and succinct one-line title for the research paper, drawing inspiration from the abstract provided. \n"

**First Input**

In [ ]:
abstract="Introducing YOLO, a revolutionary approach to object detection that breaks away from traditional methods. Instead of repurposing classifiers, YOLO tackles object detection as a regression problem, focusing on spatially separated bounding boxes and their associated class probabilities. With a single neural network, YOLO directly predicts bounding boxes and class probabilities from entire images in one swift evaluation. This unified network design allows for end-to-end optimization, streamlining the detection process. The efficiency of our architecture is remarkable. The base YOLO model achieves real-time image processing at 45 frames per second. Even a scaled-down variant, Fast YOLO, operates at an impressive 155 frames per second while doubling the mean Average Precision (mAP) compared to other real-time detectors. Though YOLO may exhibit more localization errors compared to state-of-the-art systems, its false detection rate is significantly lower, minimizing erroneous predictions in void areas. Furthermore, YOLO demonstrates remarkable adaptability, learning general object representations. It surpasses other detection methods, such as DPM and R-CNN, by a considerable margin, particularly evident when generalizing from natural images to artwork datasets like the Picasso Dataset and the People-Art Dataset."

In [ ]:
prompt=instruction+abstract

**Ouptut**

In [ ]:
text = f"<s>[INST] {prompt} [/INST]"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
temp_out=(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result = temp_out.split("[/INST]")[-1].strip()
result[:80]

'you only look once: unified, real-time object detection 1000x faster than r-cnn '

**Second Input**

In [ ]:
abstract="We introduce Adam, an algorithm for first-order gradient-based optimization of stochastic objective functions, based on adaptive estimates of lower-order moments. The method is straightforward to implement, is computationally efficient, has little memory requirements, is invariant to diagonal rescaling of the gradients, and is well suited for problems that are large in terms of data and/or parameters. The method is also appropriate for non-stationary objectives and problems with very noisy and/or sparse gradients. The hyper-parameters have intuitive interpretations and typically require little tuning. Some connections to related algorithms, on which Adam was inspired, are discussed. We also analyze the theoretical convergence properties of the algorithm and provide a regret bound on the convergence rate that is comparable to the best known results under the online convex optimization framework. Empirical results demonstrate that Adam works well in practice and compares favorably to other stochastic optimization methods. Finally, we discuss AdaMax, a variant of Adam based on the infinity norm."

In [ ]:
prompt=instruction+abstract

text = f"<s>[INST] {prompt} [/INST]"
device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
temp_out=(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result = temp_out.split("[/INST]")[-1].strip()
result[:42]

'adam: a method for stochastic optimization'